In [1]:
from webbot import Browser
from selenium import webdriver
import time
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import csv
from datetime import datetime

## ETAPE 1 : SE CONNECTER A ATOM

In [2]:
def access_atom(driver):
    """
    Accède à la plateforme Phoebus en utilisant un objet de navigateur.

    Cette fonction effectue les étapes suivantes :
    1. Navigue vers la page d'accueil de Atom Archives.
    2. Clique sur le bouton "Ouverture de session".
    3. Utilise les identifiants stockés dans le fichier 'credentials.json' pour se connecter.

    Args:
        driver: Objet de navigateur permettant d'automatiser les interactions avec la page web.
    Note:
        Assurez-vous d'avoir un fichier 'credentials.json' contenant les champs 'email' et 'password'
        pour effectuer la connexion à Phoebus.
    """
        
    driver.go_to('https://atom-archives.unil.ch/index.php')
    driver.click('Ouverture de session', tag='button')
    
    with open('credentials.json', 'r') as jsonFile:
        credentials = json.load(jsonFile)
        
    driver.type(credentials['email'], id='email')
    driver.type(credentials['password'], id='password')
    driver.click('Ouverture de session')

# LANCEMENT DE LA FONCTION
driver = Browser() #Browser pour ouvrir un navigateur
access_atom(driver)

## ETAPE 2 : ACQUISITION DES DONNEES

In [3]:
def extract_autorite(csv_path):
    """
    Extrait les noms à partir du fichier CSV contenant les noms et les données récupérées.

    Cette fonction charge un fichier CSV, remplace les valeurs NaN par une chaîne vide,
    et remplace les NaN dans la colonne 'DateExistence' par "n.c.". Ensuite, elle extrait
    les autorités de la colonne 'autoritesPHOEBUS' du DataFrame.

    Args:
        csv_path (str): Chemin vers le fichier CSV contenant les données récupérées.

    Returns:
        autorities (list): Liste des données récupérées.
    """
    
    # Charger le fichier CSV
    csv_path = '01_noticesACompleter.csv'
    df = pd.read_csv(csv_path, na_values=['NULL'], delimiter=',') #delimiter dépend de la configuration du separator du csv
    
    # Remplacer les NaN par une chaîne vide
    df = df.fillna('')
    
    # Remplacer les NULL en vide pour la colonne Historique
    df["Historique"] = df["Historique"].replace('NULL ; ', '')

    # Extraire les noms de la colonne "autoritesPHOEBUS"
    autorities = df.values.tolist()

    return autorities
    
# LANCEMENT DE LA FONCTION
csv_path = '01_noticesACompleter.csv'
autorities = extract_autorite(csv_path)
# Afficher les noms (à titre de vérification)
print(autorities)

[['https://atom-archives.unil.ch/index.php/besson-benno', 'BESSON Benno', '1922-2006', '', "Traduit de l'anglais et de l'allemand en français. Metteur en scène et acteur. - Assistant de Bertolt Brecht de 1948 à 1956. - Directeur artistique de la Volksbühne (à partir de 1969) et de la Comédie de Genève de 1982 à 1989", 'Notice ABES. Date de création : 1981-02-09 - Date de modification : 2024-04-27 à 07:35:12.', "Agence bibliographique de l'Enseignement supérieur (ABES) - Données originales récupérées sur https://www.idref.fr/026726025/id.", '', 'besson-benno_idref-026726025'], ['https://atom-archives.unil.ch/index.php/manchette-melissa', 'MANCHETTE Melissa', 's.d.', '', "D'origine grecque, traductrice de l'anglais en français. Épouse de Jean-Patrick Manchette (1962-1995)", 'Notice ABES. Date de création : 1975-11-17 - Date de modification : 2021-10-05 à 10:05:24.', "Agence bibliographique de l'Enseignement supérieur (ABES) - Données originales récupérées sur https://wwww.idref.fr/027005

## ETAPE 3 : ENTRER DANS LA NOTICE EN MODE EDITION

In [4]:
def open_autorites(csv_path):
    """
    Charge un fichier CSV, ajoute le suffixe "/edit#identityArea" à toutes les valeurs de la colonne "notice 1",
    puis extrait les noms modifiés de la colonne "notice 1".

    Args:
        csv_path (str): Chemin du fichier CSV.
    Returns:
        list: Liste des URLs renvoyant à l'édition des notices d'autorité, ouverte sur la Zone d'identification.
    """

    # Charger le fichier CSV
    df = pd.read_csv(csv_path, delimiter=',') #delimiter dépend de la configuration du separator du csv

    # Extraire les noms de la colonne "url_phoebus" et ajouter le suffixe "/edit#identityArea" à toutes les valeurs de la colonne "url_phoebus" 
    df['url_phoebus'] = df['url_phoebus'].astype(str) + '/edit#identityArea'

    # Extraire les noms de la colonne "notice 1" modifiée
    autorites_url = df['url_phoebus'].tolist()
    
    return autorites_url

# LANCEMENT DE LA FONCTION
csv_path = '01_noticesACompleter.csv'
autorites_url = open_autorites(csv_path)
#print(autorites_url)

## ETAPE 4 : OUVRIR LES ZONES ET SAISIR LES INFORMATIONS

In [5]:
def modify_autorites (driver, autorities, autorites_url):
    """
    Saisit dans les bons champs les informations récupérées dans IdRef. Cela n'implique pas les notices "renvoi"

    Cette fonction prend un objet 'driver' pour la navigation web automatisée, une liste d'autorités
    récupérées depuis IdRef, et une liste d'URLs pour les notices d'autorité. Elle ouvre chaque notice d'autorité
    avec le navigateur automatisé, ouvre toutes les Zones, saisit les informations appropriées dans les champs correspondants,
    et sauvegarde les modifications.

    Args:
        driver: Objet pour la navigation web automatisée.
        autorities (list): Liste d'autorités récupérées depuis IdRef.
        autorites_url (list): Liste d'URLs pour les notices d'autorité.

    Returns:
        None
    Notes :
        La fonction inclut les modifications automatiques effectuées par Botorité.
        A Notes de maintenance : modifier le numéro de lancement [L0.]
    """   
    try:
        
        for i, autorite_url in enumerate(autorites_url) :
            
            driver.go_to(autorite_url)

            # Ouvrir toutes les Zones
            driver.click('Zone du contrôle')   
            driver.execute_script("window.scrollTo(0,0);")
            time.sleep(2)
            driver.click('Zone de description')
            # Récupérer le contenu HTML depuis la page
            content = driver.get_page_source()
            soup = BeautifulSoup(content, features='lxml')
            time.sleep(2)
            
            ## ZONE IDENTIFICATION
            # Modification du type d'entité
            driver.click(tag='select', id='entityType')
            driver.click('Personne', tag='option')

            # saisi des autres formes de nom
            other_name_value = autorities[i][7]
            if driver.exists(id='otherName_new'):
                # Récupérer le contenu HTML de la page
                html_content = driver.get_page_source()
                # Analyser le HTML avec BeautifulSoup
                soup = BeautifulSoup(html_content, 'html.parser')
                # Trouver l'élément input avec l'ID otherName_new
                input_element = soup.find('input', id='otherName_new')
                # Si l'élément input est trouvé, cela signifie que le champ est déjà saisi, donc ne fait rien
                if input_element:
                    # Vérifier si le champ est vide
                    if not input_element.get('value'):
                        # Si le champ est vide, saisir la valeur
                        driver.type(other_name_value, into='otherName_new')
                        time.sleep(1)
            else:
                print(f"ERREUR pour {autorities[i][1]} : Autre(s) forme(s) de nom n'a pas été trouvée et saisie")   
                
            ## ZONE DESCRIPTION
            # saisie des dates d'existence, de l'historique et des fonctions
            existence_value = autorities[i][2]
            history_value = autorities[i][4]
            functions_value = autorities[i][3]
            
            if driver.exists(id='datesOfExistence'):
                 # Si l'élément existe et est visible, alors saisir la valeur
                driver.type(existence_value, id='datesOfExistence')
            else:
                print(f"ERREUR pour {autorities[i][1]} : Dates d'existence n'ont pas été trouvées et saisies")
                
            if driver.exists(id='history'):
                driver.type(history_value, id='history')
            else:
                print(f"ERREUR pour {autorities[i][1]} : Historique n'a pas été trouvée et saisie")          
            if driver.exists(id='functions'):
                
                driver.type(functions_value, id='functions')
            else:
                print(f"ERREUR pour {autorities[i][1]} : Fonctions et activités n'ont pas été trouvées et saisies") 
                        
            time.sleep(1)
            
            ## ZONE DE CONTROLE

            # saisie de l'identifiant basé sur celui d'idRef
            if driver.exists(id='descriptionIdentifier'):
                driver.type(autorities[i][8], id='descriptionIdentifier')
            else:
                print(f"ERREUR pour {autorities[i][1]} : Identifiant de la description n'a pas été trouvée et saisie")
            
            # Modification de l'entretien du dépôt
            if driver.exists(id='maintainingRepository'):
                driver.type('Centre des littératures en Suisse romande', id='maintainingRepository')
                time.sleep(4)
                driver.press(driver.Key.ENTER)
            else:
                print(f"ERREUR pour {autorities[i][1]} : Entretien du dépôt de la description n'a pas été trouvé et saisi")
            
            
            # Modification du statut
            driver.click(tag='select', id='descriptionStatus')
            driver.click('Final', tag='option')
            
            # saisie de l'historique de révision et de la référence
            
            if driver.exists(id='revisionHistory'):
                driver.type(autorities[i][5], id='revisionHistory')
            else:
                print(f"ERREUR pour {autorities[i][1]} : Dates de production et de révision n'ont pas été trouvées et saisies")
            
            if driver.exists(id='sources'):
                driver.type(autorities[i][6], id='sources')
            else:
                print(f"ERREUR pour {autorities[i][1]} : Sources n'ont pas été trouvées et saisies")
            
            # Notes de maintenance            
            today_date = datetime.today().strftime('%Y-%m-%d à %H:%M:%S')
            maintenance_text = f"""Saisie automatique [LaTempDemo] effectuée le : """ + today_date
            if driver.exists(id='maintenanceNotes'):
                driver.type(maintenance_text, id='maintenanceNotes')
            else:
                print(f"ERREUR pour {autorities[i][1]} : Notes de maintenance n'ont pas été trouvées et saisies")
      
            
            # scroller vers le bas puis, sauvegarder la saisie
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(2)

            driver.click('Sauvegarder', tag='input')
            
            print(f"({i+1}) Opération de saisie terminée pour {autorities[i][1]} : {autorities[i][0]}")
        
    except Exception as e:
        print(f"Etape 9 : Erreur lors de la modification des notices d'autorité : {e}")
        
# LANCEMENT DE LA FONCTION        
modify_autorites(driver, autorities, autorites_url)


(1) Opération de saisie terminée pour BESSON Benno : https://atom-archives.unil.ch/index.php/besson-benno
(2) Opération de saisie terminée pour MANCHETTE Melissa : https://atom-archives.unil.ch/index.php/manchette-melissa
(3) Opération de saisie terminée pour MANDEL Ernest : https://atom-archives.unil.ch/index.php/mandel-ernest


## LANCEMENT DES FONCTIONS

In [6]:
# ETAPE 1
driver = Browser()
access_atom(driver)

# ETAPE 2
csv_path = '01_noticesACompleter.csv'
autorities = extract_autorite(csv_path)

# ETAPE 3
csv_path = '01_noticesACompleter.csv'
autorites_url = open_autorites(csv_path)

# ETAPE 4
modify_autorites(driver, autorities, autorites_url)

Etape 9 : Erreur lors de la modification des notices d'autorité : Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.91)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.16.0 x86_64)

